# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
# replace the code below with your pulse rate algorithm.
import glob

import numpy as np
import scipy as sp
import scipy.signal
import scipy.io

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, passband=(40/60, 200/60), fs=125):
    """Bandpass filter the signal between 40bpm and 200bpm"""
    b, a = sp.signal.butter(3, passband, btype="bandpass", fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def RunPulseRateAlgorithm(data_fl, ref_fl, fs=125, conf_freq = 5/60, minFreq =(40/60), maxFreq =(240/60)):
    # Load data using LoadTroikaDataFile and the reference/ground truth data
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_data = sp.io.loadmat(ref_fl)['BPM0'].squeeze()

    #Apply bandpass to filter the signals that are out of range
    ppg = BandpassFilter(ppg)
    accx = BandpassFilter(accx)
    accy = BandpassFilter(accy)
    accz = BandpassFilter(accz)
    accmag = np.sqrt(accx**2 + accy**2 + accz**2)
      
    
    #Calculate spectrogram of the signals
    acc_f, _, acc_spec = sp.signal.spectrogram(accmag, fs=fs, nperseg=8*fs, noverlap=6*fs)
    ppg_f, _, ppg_spec = sp.signal.spectrogram(ppg, fs=fs, nperseg=8*fs, noverlap=6*fs)
       

    #keep realistic frequencies / discard the ones outside of the set limits
    valid_idx = (ppg_f >= minFreq) & (ppg_f <= maxFreq)
    ppg_spec[~valid_idx,:] = 0
    

    # Compute pulse rate estimates and estimation confidence.
    errors = []
    confidence = []
    last_ppg_maxf = ppg_f[np.argmax(ppg_spec[:,0])] #We're keeping track of the last estimate for smoothing
    
    for acc_window, ppg_window, target in zip(acc_spec.T, ppg_spec.T, ref_data):
        
        #Find peak/dominant frequencies
        #ACC - peak/dominant frequencies 
        acc_i = np.argmax(acc_window)
        acc_maxf = acc_f[acc_i]
        
        #PPG peak/dominant frequencies
        ppg_k_i = np.argsort(ppg_window)[::-1][:5]
        ppg_k_maxf = ppg_f[ppg_k_i]
        
   
        #Order frequencies by the closest to latest ppg_max frequency 
        new_idx = np.argsort(np.abs(ppg_k_maxf - last_ppg_maxf))
        ppg_k_maxf = ppg_k_maxf[new_idx]
       
        #if the ppg frequency is equal to the acc one select a different one.   
        for i in ppg_k_maxf:
            if acc_maxf == i:
                continue
            else:
                ppg_maxf = i
                break
        
        #Calculate the error 
        est_hr = ppg_maxf * 60
        error = est_hr - target
        errors += [error] #Add the individual error value to the total errors
        
        #Calculate the confidence       
        conf_window = (ppg_f > ppg_maxf - conf_freq) & (ppg_f < ppg_maxf + conf_freq)
        conf = np.sum(ppg_window[conf_window]) / np.sum(ppg_window)
        confidence += [conf] #Add the individual confidence value to the confidence total
        
        #Update last ppg with the maxfreq
        last_ppg_maxf = ppg_maxf
        
         
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.  
    return errors, confidence